# ICMA395 Project: Finding Magic Squares Using Heuristic Methods

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import random
import copy
import math
import seaborn as sns
import matplotlib.pyplot as plt
from collections import deque

### 2-step process:

1. Find a semi magic square
2. Make moves that preserve row and column sums to find a full magic square

### Perturbation methods:

1. Swap random entries ('Naive')
2. Swap two neighboring entries 

## Setup

In [2]:
def make_matrix(sol, order):
    ans = [[] for _ in range(order)]
    j = 0
    for i in range(len(sol)):
        ans[j].append(sol[i])
        if (i+1)%order == 0:
            j+=1
    return ans

def random_sol(order):
    lst = [i+1 for i in range(order**2)]
    random.shuffle(lst)
    return lst

def matrix_to_sol(matrix):
    sol = []
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            sol.append(matrix[i][j])
    return sol

def cost1(sol, n):
    S = int(n*(n**2 + 1)/2)
    column_sums = np.sum(sol, axis=0)
    row_sums = np.sum(sol, axis=1)
    # diagonal_sums = np.array([np.trace(sol), np.trace(np.fliplr(sol))])
    return np.sum(abs(S - column_sums)) + np.sum(abs(S-row_sums))

def cost2(sol, n):
    S = int(n*(n**2 + 1)/2)
    return abs(S - np.trace(sol)) + abs(S - np.trace(np.fliplr(sol)))

def cost(sol, n):
    return cost1(sol, n) + cost2(sol, n)

def adj_swap(sol, order):
    new_sol = copy.deepcopy(sol)
    i1 = random.randint(0, order-1)
    j1 = random.randint(0, order-1)
    temp = new_sol[i1][j1]
    neighbors = ['left','right','above','below']
    if i1 == 0:
        neighbors.remove('above')
    if i1 == order-1:
        neighbors.remove('below')
    if j1 == 0:
        neighbors.remove('left')
    if j1 == order-1:
        neighbors.remove('right')
    choice = random.choice(neighbors)
    if choice == 'left':
        new_sol[i1][j1] = new_sol[i1][j1-1]
        new_sol[i1][j1-1] = temp
    elif choice == 'right':
        new_sol[i1][j1] = new_sol[i1][j1+1]
        new_sol[i1][j1+1] = temp
    elif choice == 'above':
        new_sol[i1][j1] = new_sol[i1-1][j1]
        new_sol[i1-1][j1] = temp
    elif choice == 'below':
        new_sol[i1][j1] = new_sol[i1+1][j1]
        new_sol[i1+1][j1] = temp
    return new_sol

def gen_starting_states(order, n_trials):
    return [random_sol(order) for _ in range(n_trials)]

## Local Search

In [10]:
def naive_local_search(n_trials, n_iterations, order):
    starting_sols = gen_starting_states(order, n_trials)
    for trial in range(n_trials):
        current_sol = starting_sols[trial]
        current_cost = cost1(make_matrix(current_sol, order), order)
        for i in range(n_iterations):
            idx1 = random.randint(0, order**2-1)
            idx2 = random.randrange(0, order**2-1)
            new_sol = current_sol.copy()
            temp = new_sol[idx1]
            new_sol[idx1] = new_sol[idx2]
            new_sol[idx2] = temp
            new_cost = cost1(make_matrix(new_sol, order), order)
            if new_cost < current_cost:
                current_cost = new_cost
                current_sol = new_sol
    return current_sol

def local_search(n_trials, n_iterations, order):
    starting_sols = gen_starting_states(order, n_trials)
    for trial in range(n_trials):
        current_sol = make_matrix(starting_sols[trial], order)
        current_cost = cost1(current_sol, order)
        for i in range(n_iterations):
            new_sol = adj_swap(current_sol, order)
            new_cost = cost1(new_sol, order)
            if new_cost < current_cost:
                current_cost = new_cost
                current_sol = new_sol
    return current_sol

### Usage

In [15]:
n = 3
sol = make_matrix(naive_local_search(1, 100, n), n)
cost_val = cost1(sol, n)

sol, cost_val

([[7, 1, 8], [5, 4, 6], [3, 9, 2]], 4)

## Simulated Annealing

In [22]:
def naive_sa(n_trials, alpha, n_iterations, order):
    starting_sols = gen_starting_states(order, n_trials)
    for trial in range(n_trials):
        current_sol = starting_sols[trial]
        current_cost = cost1(make_matrix(current_sol, order), order)
        if order == 3:
            T0 = 100
        elif order == 4:
            T0 = random.randint(100, 1000)
        else: T0 = 1000
        t = T0
        i = 0
        accepted = 0
        total = 0
        while t > 0 and i < n_iterations:
            idx1 = random.randint(0, order**2-1)
            idx2 = random.randrange(0, order**2-1)
            new_sol = current_sol.copy()
            temp = new_sol[idx1]
            new_sol[idx1] = new_sol[idx2]
            new_sol[idx2] = temp
            new_cost = cost1(make_matrix(current_sol, order), order)
            dy = new_cost - current_cost
            if dy < 0:
                current_sol = new_sol
                current_cost = new_cost
            else:
                total += 1
                if random.uniform(0, 1) < np.exp(-dy/t):
                    accepted += 1
                    current_sol = new_sol
                    current_cost = new_cost
            i = i+1
            t = alpha*t
    return current_sol

def sa(n_trials, alpha, n_iterations, order):
    starting_sols = gen_starting_states(order, n_trials)
    for trial in range(n_trials):
        current_sol = make_matrix(starting_sols[trial], order)
        current_cost = cost1(current_sol, order)
        if order == 3:
            T0 = 100
        elif order == 4:
            T0 = random.randint(100, 1000)
        else: T0 = 1000
        t = T0
        i = 0
        while t > 0 and i < n_iterations:
            new_sol = adj_swap(current_sol, order)
            new_cost = cost1(new_sol, order)
            dy = new_cost - current_cost
            if dy < 0:
                current_sol = new_sol
            else:
                if random.uniform(0, 1) < np.exp(-dy/t):
                    current_sol = new_sol
            t = alpha*t
            i += 1
    return current_sol

### Usage

In [23]:
n = 3
sol = make_matrix(naive_sa(1, 0.95, 100, n), n)
cost_val = cost1(sol, n)

sol, cost_val

([[4, 8, 1], [5, 7, 3], [2, 6, 9]], 16)

## Tabu Search

In [26]:
def naive_tabu(n_trials, k, V, n_iterations, order):
    starting_sols = gen_starting_states(order, n_trials)
    for trial in range(n_trials):
        current_sol = starting_sols[trial]
        tabu_list = deque()
        freq = dict()
        global_best = float('inf')
        global_best_sol = current_sol
        for i in range(n_iterations):
            neighbors = []
            for j in range(V):
                idx1 = random.randint(0, order**2-1)
                idx2 = random.randrange(0, order**2-1)
                while idx1 == idx2:
                    idx2 = random.randrange(0, order**2-1)
                pair = [idx1, idx2]
                pair.sort()
                pair = tuple(pair)
                neighbor = current_sol.copy()
                temp = neighbor[idx1]
                neighbor[idx1] = neighbor[idx2]
                neighbor[idx2] = temp
                tabu = False
                if pair in tabu_list:
                    tabu = True
                if pair not in freq.keys():
                    freq[pair] = 0
                new_cost = cost1(make_matrix(neighbor, order), order)
                neighbor_matrix = make_matrix(neighbor, order)
                if (i+1)%30 == 0 and new_cost > cost1(make_matrix(current_sol, order), order):
                    new_cost += 5*freq[pair]
                neighbors.append((neighbor_matrix, pair, tabu, new_cost))
            neighbors.sort(key=lambda a: a[3])
            idx = 0
            all_tabu = False
            while neighbors[idx][2]:
                if neighbors[idx][3] < global_best:
                    break
                elif idx < V - 1:
                    idx += 1
                elif idx == V - 1:
                    all_tabu = True
                    break
            if not all_tabu:
                curr = neighbors[idx][0]
                attribute = neighbors[idx][1]
                tabu_list.append(attribute)
                freq[attribute] += 1
                if len(tabu_list) > k:
                    tabu_list.popleft()
                local_best = neighbors[idx][3]
                if local_best < global_best:
                    global_best = local_best
                    global_best_sol = curr
    return global_best_sol

### Usage

In [33]:
n = 3
sol = naive_tabu(1, 4, 10, 100, n)
cost_val = cost1(sol, n)

sol, cost_val

([[3, 9, 5], [6, 4, 2], [8, 1, 7]], 10)

## Genetic Algorithm

### Crossover methods

In [34]:
def pmx(p1, p2, idx):
    left = []
    right = p2[idx:]
    for char in p1[0: idx]:
        if char in right:
            mapping = p1[np.where(np.array(p2) == char)[0][0]]
            while mapping in right:
                mapping = p1[np.where(np.array(p2) == mapping)[0][0]]
            left.append(mapping)
        else:
            left.append(char)
    return left + right

def ox(p1, p2, idx):
    left = p1[0: idx]
    right = ''
    for char in p2:
        if char not in left:
            right += char
    return left + right

def cx(p1, p2):
    size = len(p1)
    child = [''] * size
    cycle = []
    while '' in child:
        cycle_start = child.index('')
        index = cycle_start
        new_cycle = []
        while True:
            new_cycle.append(index)
            index = p2.index(p1[index])
            if index == cycle_start:
                break
        if len(cycle) % 2 == 0:
            for i in new_cycle:
                child[i] = p1[i]
        else:
            for i in new_cycle:
                child[i] = p2[i]
        cycle.extend(new_cycle)
    return ''.join(child)

def mutate(child, idx1, idx2):
    temp = child[idx1]
    child[idx1] = child[idx2]
    child[idx2] = temp
    return child

### Algorithm

In [80]:
def ga(starting_gen, n_gens, pop_size, p_cross, p_mutate, order):
    current_gen = starting_gen
    for gen in range(n_gens):
        new_gen = []
        current_gen.sort(key=lambda x: 1/(1+cost(make_matrix(x, order), order)))
        fitness_list = [1/(1+cost(make_matrix(ind, order), order)) for ind in current_gen]
        new_gen.append(current_gen[-1])
        best_fitness = max(fitness_list)
        average_fitness = np.mean(fitness_list)
        children = []
        while len(children) < pop_size:
            temp_fitness_list = fitness_list.copy()
            running_total = [sum(temp_fitness_list[0:i+1]) for i in range(len(temp_fitness_list))]
            temp_current_gen = current_gen.copy()
            num1 = random.uniform(0, sum(fitness_list))
            for i in range(len(temp_fitness_list)):
                if running_total[i] >= num1:
                    parent1 = temp_current_gen.pop(i)
                    temp_fitness_list.pop(i)
                    break
            running_total = [sum(temp_fitness_list[0:i+1]) for i in range(len(temp_fitness_list))]
            num2 = random.uniform(0, sum(temp_fitness_list))
            for i in range(len(temp_fitness_list)):
                if running_total[i] >= num2:
                    parent2 = temp_current_gen.pop(i)
                    temp_fitness_list.pop(i)
                    break
            idx = random.randint(1, order**2 - 2)
            cross = random.uniform(0, 1)
            if cross <= p_cross:
                choice = random.uniform(0, 1)
                if choice <= 0.5:
                    child1 = pmx(parent1, parent2, idx)
                    children.append(child1)
                else:
                    child1 = pmx(parent2, parent1, idx)
                    children.append(child1)
            for child in children:
                m = random.uniform(0, 1)
                if m <= p_mutate:
                    idx1 = random.randint(0, len(child)-1)
                    idx2 = random.randint(0, len(child)-1)
                    child = mutate(child, idx1, idx2)
            candidates = current_gen + children
            candidates.sort(key=lambda x: 1/(1+cost(make_matrix(x, order), order)), reverse=True)
            i=0
            while len(new_gen) < pop_size:
                new_gen.append(candidates[i])
                i = i + 1
            current_gen = new_gen
    return current_gen

### Semi-magic -> Magic

In [39]:
def get_semi_magic(n_trials, n_iterations, order):
    starting_sols = gen_starting_states(order, n_trials)
    semi_magic = []
    for trial in range(n_trials):
        current_sol = starting_sols[trial]
        current_cost = cost(make_matrix(current_sol, order), order)
        for i in range(n_iterations):
            idx1 = random.randint(0, order**2-1)
            idx2 = random.randrange(0, order**2-1)
            new_sol = current_sol.copy()
            temp = new_sol[idx1]
            new_sol[idx1] = new_sol[idx2]
            new_sol[idx2] = temp
            new_cost = cost1(make_matrix(new_sol, order), order)
            if new_cost < current_cost:
                current_cost = new_cost
                current_sol = new_sol
            if current_cost == 0:
                semi_magic.append(current_sol)
                break
    return semi_magic

In [45]:
semi_magic_3 = get_semi_magic(100, 10000, 3)
semi_magic_4 = get_semi_magic(100, 10000, 4)
semi_magic_5 = get_semi_magic(100, 10000, 5)
semi_magic_6 = get_semi_magic(100, 10000, 6)
semi_magic_7 = get_semi_magic(100, 10000, 7)
semi_magic_8 = get_semi_magic(100, 10000, 8)
semi_magic_9 = get_semi_magic(100, 10000, 9)